# Per session and per user analysis
Analysis of users.



# Table of Contents

1.  [Preparation](#preparation)

2.  [Function tests](#functests)

3. [User metrics checks](#checkusermetrics)

# Preparation
<a id=preparation />

In [ ]:
%run "../Functions/3. Per session and per user analysis.ipynb"

In [ ]:
df152.head()

# Per-session analysis

In [ ]:


testSessionId = "fab3ea03-6ff1-483f-a90a-74ff47d0b556"

perSession = df152[df152['type']=='reach'].loc[:,perSessionRelevantColumns]
perSession = perSession[perSession['sessionId']==testSessionId]
perSession = perSession[perSession['section'].str.startswith('tutorial', na=False)]
perSession

In [ ]:
allSessions = getAllSessions( df152, True )
allSessions.head()

In [ ]:
allSessions[allSessions['sessionId']==testSessionId]

In [ ]:
allSessions[allSessions['userId']=='"e2f8d5e4-cccd-4d1a-909b-c9c92f6b83c1"']

# Per user analysis

In [ ]:
# English-speaking user who answered the questionnaire - cf 'Google form analysis.ipynb'.
localplayerguid = '"8d352896-a3f1-471c-8439-0f426df901c1"'
#localplayerguid = '"7037c5b2-c286-498e-9784-9a061c778609"'
#localplayerguid = '"5c4939b5-425b-4d19-b5d2-0384a515539e"'
#localplayerguid = '"7825d421-d668-4481-898a-46b51efe40f0"'
#localplayerguid = '"acb9c989-b4a6-4c4d-81cc-6b5783ec71d8"'
localplayerguid

In [ ]:
perUserRelevantColumns = ['sessionId', 'serverTime', 'section']

In [ ]:
sessionsList = getAllSessionsOfUser(df152, localplayerguid, True)
sessionsList

In [ ]:
# List all 'reach' events with those sessionIds.
perUser = df152[df152['type']=='reach'].loc[:,perUserRelevantColumns]
perUser = perUser[perUser['sessionId'].isin(sessionsList['sessionId'])]
perUser = perUser[perUser['section'].str.startswith('tutorial', na=False)]
perUser.describe()
perUser.head()

# Common analysis

#### Switch here between users and sessions.

In [ ]:
#sectionsList = perSession
sectionsList = perUser

In [ ]:
timedSections = sectionsList.groupby("section").agg({ "serverTime": np.min })
timedSections.head()

In [ ]:
timedSections["firstReached"] = pd.to_datetime(timedSections["serverTime"])
timedSections.head()

In [ ]:
timedSections["firstCompletionDuration"] = timedSections["firstReached"].diff()
timedSections

#### getCheckpointsTimes tinkering

In [ ]:
testUser = unsafeGetRedMetricsGUIDFormat(getRandomGFormGUID())
testSession = getRandomSessionGUID( _userId = testUser )

In [ ]:
timedSections1 = getCheckpointsTimes(testSession)
timedSections1

In [ ]:
sessionId = testSession
timedSections = []

# Returns a given session's checkpoints, the first server time at which they were reached, and completion time
#def getCheckpointsTimes( sessionId ):
perSession = reachEvents[reachEvents['sessionId']==sessionId]
perSession = perSession[perSession['section'].str.startswith('tutorial', na=False)]

timedSections = pd.DataFrame(data=0, columns=timedSectionsReachedColumns,index=timedSectionsIndex)
timedSections["serverTime"] = perSession.groupby("section").agg({ "serverTime": np.min })
timedSections["firstReached"] = pd.to_datetime(timedSections["serverTime"])
timedSections["firstCompletionDuration"] = timedSections["firstReached"].diff()
#if("tutorial1.Checkpoint00" in timedSections.index):
#    timedSections.loc["tutorial1.Checkpoint00","firstCompletionDuration"] = 0

timedSections = timedSections.fillna(0)

timedSections

In [ ]:
len(timedSections)

In [ ]:
chapter = "tutorial1.Checkpoint01"
time = ''
if(not chapter in timedSections.index):    
    print("no timed sections")
else:
    time = timedSections.loc[chapter,"firstCompletionDuration"]
time

In [ ]:
timedSections1 == timedSections

In [ ]:
reachEvents.iloc[0,0]

# Function tests
<a id=functests />

In [ ]:
#'"7412a447-8177-48e9-82c5-cb31032f76a9"': didn't answer
testUser = unsafeGetRedMetricsGUIDFormat(getRandomGFormGUID())
testResult = getUserDataVector(testUser)
print(testUser)
testResult

In [ ]:
testResult[testUser]['death']

In [ ]:
testResult = getUserDataVector('"e2f8d5e4-cccd-4d1a-909b-c9c92f6b83c1"')
testResult

In [ ]:
testResult = getUserDataVector('"8d352896-a3f1-471c-8439-0f426df901c1"')
testResult

In [ ]:
gformNotEnough = []
print(gformNotEnough)

gformNotEnough.append(5)
print(gformNotEnough)

gformNotEnough = pd.Series(gformNotEnough)
print(gformNotEnough)

gformNotEnough = np.array([])
print(gformNotEnough)

gformNotEnough = np.append(gformNotEnough, [5])
print(gformNotEnough)

gformNotEnough = pd.Series(gformNotEnough)
print(gformNotEnough)

testNonVal = pd.Series(['tutorial1.Checkpoint13'])

#### getDiscrepancyGameGForm tinkering

In [ ]:
randomguid = getRandomRedMetricsGUID()
randomguid

In [ ]:
gformNonVal = getNonValidatedCheckpoints(randomguid)
gformNonVal

In [ ]:
gformVal = getValidatedCheckpoints(randomguid)
gformVal

In [ ]:
gameVal = getUserCheckpoints( randomguid )
gameVal

#sorted, unique values in series1 that are not in series2
#np.setdiff1d(series1.values, series2.values)

#user has answered questions whose answer they haven't seen in the game
gameNotEnough = pd.Series(np.setdiff1d(gformVal.values, gameVal.values))


In [ ]:

#user has not answered questions whose answer they have seen in the game
gformNotEnough = []
maxGameVal = ''

if gameVal.values.size!=0:
    gameVal.values.max()
for nonVal in gformNonVal.values:
    if nonVal >= maxGameVal:
        gformNotEnough.append(nonVal)    
gformNotEnough = pd.Series(gformNotEnough)

getDiscrepancyGameGForm( randomguid )

In [ ]:
getUserCheckpoints(localplayerguid)

In [ ]:
test = getValidatedCheckpoints(localplayerguid)
test

max = test.values.max()
max

getNonValidatedCheckpoints(localplayerguid)

testlocalplayerguid = '"7412a447-8177-48e9-82c5-cb31032f76a9"'

test = pd.DataFrame({
                        'section' : ['tutorial1.Checkpoint00', 'tutorial1.Checkpoint01', 'tutorial1.Checkpoint02'],
                        'serverTime' : ['0', '1', '2'],
                        'firstReached' : ['0', '1', '2'],
                        'firstCompletionDuration' : ['0', '1', '2'],
                    })
test
#pd.DataFrame({  'A' : 1.,
#                'B' : pd.Timestamp('20130102'),
#                'C' : pd.Series(1,index=list(range(4)),dtype='float32'),
#                'D' : np.array([3] * 4,dtype='int32'),
#                'E' : pd.Categorical(["test","train","test","train"]),
#                'F' : 'foo' })

In [ ]:
testSr = pd.Series([])
testSr.values.size

In [ ]:
checkpointTimes = getCheckpointsTimes('"958a0e85-1634-4559-bce6-d6af28b7e649"')
checkpointTimes

In [ ]:
type(checkpointTimes)

In [ ]:
testDF = pd.DataFrame({}, columns=checkpointTimes.columns)
testDF

In [ ]:
testDF['serverTime']['testIndex'] = np.nan
testDF

In [ ]:
testDF = testDF.add({'serverTime': 'a', 'firstReached': 'b', 'firstCompletionDuration': 'c'})
testDF

In [ ]:
othersessionId = "c430c60c-d7cb-4fc5-893b-dce440696caf"
checkpointTimes = getCheckpointsTimes(othersessionId)
checkpointTimes

In [ ]:
checkpointTimes.loc[checkpointTimes.index[0]]

In [ ]:
checkpointTimes.loc[checkpointTimes.index[0]].name

In [ ]:
reachEvents.head()

In [ ]:
perSession = reachEvents[reachEvents['sessionId']==othersessionId]
perSession.head()

In [ ]:
perSession = perSession[perSession['section'].str.startswith('tutorial', na=False)]
perSession.head()

In [ ]:
timedSections = perSession.groupby("section").agg({ "serverTime": np.min })
timedSections.head()

In [ ]:
timedSections["firstReached"] = pd.to_datetime(timedSections["serverTime"])
timedSections.head()

In [ ]:
timedSections["firstCompletionDuration"] = timedSections["firstReached"].diff()
timedSections.head()

In [ ]:
pd.melt(timedSections).head(1)

In [ ]:
timedSections.unstack().reset_index().drop('level_0', axis=1).head(1)

In [ ]:
timedSections.columns

In [ ]:
timedSections.index

In [ ]:
#timedSections[timedSections['section']=='tutorial1.Checkpoint00']
#timedSections['section']
#timedSections.unstack().reset_index().drop('level_0', axis=1)

In [ ]:
timedSections['serverTime']['tutorial1.Checkpoint00']

In [ ]:
timedSections['firstReached']['tutorial1.Checkpoint00']

In [ ]:
testTimeStamp = pd.Timestamp('2017-01-12 11:00:15.511000')
testTimeStamp

In [ ]:
timedSections.loc['tutorial1.Checkpoint00']

In [ ]:
timedSections.loc['tutorial1.Checkpoint00']['serverTime']

In [ ]:
for checkpoint in timedSections.index:
    print(checkpoint)

In [ ]:
#timedSections = []

#times = getCheckpointsTimes( sessionId )

for checkpoint in timedSections.index:
    if testTimeStamp < timedSections['firstReached'][checkpoint]:
        print(checkpoint)

In [ ]:
timedSections.empty

In [ ]:
checkpointTimes.index

In [ ]:
checkpointTimes.columns

In [ ]:
testTimedSections = pd.DataFrame(data=None, columns=timedSectionsReachedColumns,index=timedSectionsIndex)
testTimedSections

In [ ]:
timedSectionsReachedColumns

#### getCheckpointsTimesUser tinkering

In [ ]:
# incomplete game
_userId = '"dfe8f036-8641-4d6c-8411-8a8346bb0402"'
#_userId = getRandomRedMetricsGUID()
_sessionsList = []
_rmDF = df152

# Returns a given user's checkpoints, the first server time at which they were reached, and completion time
#def getCheckpointsTimesUser( _userId, _sessionsList = [], _rmDF = df152 ):
# List of associated sessions
if( len(_sessionsList) == 0):
    _sessionsList = getAllSessionsOfUser( _rmDF, _userId, True )

# Call getCheckpointsTimes on all sessions associated with user,
# then merge by taking oldest checkpoint completion
_timedSections = pd.DataFrame(data=0, columns=timedSectionsReachedColumns,index=timedSectionsIndex)
_timedSections["serverTime"] = ''
_timedSections["firstReached"] = pd.Timestamp(0)
_timedSections["firstCompletionDuration"] = pd.Timedelta.max

# merge
# for each checkpoint reached, update if necessary
for _sessionId in _sessionsList['sessionId']:        
    _thisSessionTimes = getCheckpointsTimes( _sessionId )

    for _checkpointName in _thisSessionTimes.index:
        if ((_thisSessionTimes.loc[_checkpointName, 'firstReached'] != pd.Timestamp(0))
            and
            ((_timedSections.loc[_checkpointName, 'firstReached'] == pd.Timestamp(0))
            or (_timedSections.loc[_checkpointName, 'firstReached'] > _thisSessionTimes.loc[_checkpointName, 'firstReached']))
           ):
            _timedSections.loc[_checkpointName, 'serverTime'] = _thisSessionTimes.loc[_checkpointName, 'serverTime']
            _timedSections.loc[_checkpointName, 'firstReached'] = _thisSessionTimes.loc[_checkpointName, 'firstReached']
            _timedSections.loc[_checkpointName, 'firstCompletionDuration'] = _thisSessionTimes.loc[_checkpointName, 'firstCompletionDuration']

_timedSections

print('second pass')
            
previous = ''
for checkpointName in thisSessionTimes.index:
    if(checkpointName != "tutorial1.Checkpoint00"):
        if(
            timedSections.loc[previous,"firstReached"] != pd.Timestamp(0)
            and
            timedSections.loc[checkpointName,"firstReached"] != pd.Timestamp(0)
        ):
            timedSections.loc[checkpointName,"firstCompletionDuration"] =\
            timedSections.loc[checkpointName,"firstReached"] - timedSections.loc[previous,"firstReached"]
    previous = checkpointName
#timedSections["firstCompletionDuration"] = timedSections["firstReached"].diff()
timedSections

In [ ]:
timedSections["test"] = range(0,len(timedSections["firstReached"]))

In [ ]:
timedSections["test"].diff()

In [ ]:
timedSections["firstReached"].fillna(pd.Timestamp.max)

In [ ]:
timedSections["firstReached"][:14].diff()

In [ ]:
nanVar = np.nan
if(np.isnan(nanVar)):
    print('np.isnan(nan)')
else:
    print('not np.isnan(nan)')
if(nanVar == np.nan):
    print('np.nan == np.nan')
else:
    print('np.nan != np.nan')

In [ ]:
natVar = np.datetime64('NaT')
#if(np.isnat(natVar)):
#    print('np.isnat(NaT)')
#else:
#    print('not np.isnat(NaT)')
if(natVar == np.datetime64('NaT')):
    print('np.NaT == np.NaT')
else:
    print('np.NaT != np.NaT')

In [ ]:
checkpointName = 'tutorial1.Checkpoint00'
timedSections['firstReached'][checkpointName]

In [ ]:
timedSections

In [ ]:
checkpointName

In [ ]:
checkpointTimesUser = getCheckpointsTimesUser(testlocalplayerguid)
checkpointTimesUser

In [ ]:
checkpointTimesUser = getCheckpointsTimesUser(testlocalplayerguid)
checkpointTimesUser

In [ ]:
checkpointTimesUser['firstCompletionDuration']

In [ ]:
testTimedSections = pd.DataFrame(data=None, columns=timedSectionsReachedColumns,index=timedSectionsIndex)
#testTimedSections['firstReached']['tutorial1.Checkpoint01'] = pd.Timestamp('20130102')
testTimedSections["firstCompletionDuration"] = testTimedSections["firstReached"].diff()
testTimedSections

In [ ]:
# user "e2f8d5e4-cccd-4d1a-909b-c9c92f6b83c1"
# session testSessionId
getDeaths(testSessionId)
testResult = getDeathsUser('"e2f8d5e4-cccd-4d1a-909b-c9c92f6b83c1"')
testResult
testResult.index
#localplayerguid
#"8bd5d335-d20e-4db5-9979-feb668ba7c71"
getDeaths("8bd5d335-d20e-4db5-9979-feb668ba7c71")
getDeathsUser(localplayerguid)

In [ ]:
eventSectionsColumns

In [ ]:
perSessionRelevantColumns

In [ ]:
sectionsEvents = getSectionsEvents('death', testSessionId)
sectionsEvents

In [ ]:
testResult = getUserSectionsEvents('death', '"e2f8d5e4-cccd-4d1a-909b-c9c92f6b83c1"')
testResult

In [ ]:
getUserSectionsEventsTotal('death', '"e2f8d5e4-cccd-4d1a-909b-c9c92f6b83c1"')

#### getUserCraftEventsTotal tinkering

In [ ]:
# craftEventCodes = list(["equip","unequip","add","remove"])
eventCode = 'equip'
userId = getRandomRedMetricsGUID()
sessionsList=[]
_rmDF = df152

#def getUserCraftEventsTotal( eventCode, userId, sessionsList=[], _rmDF = df152 ):
if(len(sessionsList) == 0):
    sessionsList = getAllSessionsOfUser( _rmDF, userId, True )

result = 0
    
if eventCode in craftEventCodes:
    eventType = craftEventsColumns['eventType'][eventCode]
    events = _rmDF[_rmDF['type']==eventType]
    events = events[events[craftEventsColumns['column'][eventCode]].notnull()]
    perSession = events[events['sessionId'].isin(sessionsList['sessionId'])]
    result = len(perSession)
else:
    print("incorrect event code '" + eventCode + "'")
result, userId

#### getUserEventsTotal tinkering

In [ ]:
eventType = 'death'
#userId = '"e2f8d5e4-cccd-4d1a-909b-c9c92f6b83c1"'
userId = getRandomRedMetricsGUID()
sessionsList=[]
_rmDF = df152

#def getUserEventsTotal( eventType, userId, sessionsList=[], _rmDF = df152 ):
if(len(sessionsList) == 0):
    sessionsList = getAllSessionsOfUser( _rmDF, userId, True )

sessionEvents = _rmDF[_rmDF['type']==eventType]
perSession = sessionEvents[sessionEvents['sessionId'].isin(sessionsList['sessionId'])]
len(perSession)

#### getSessionDataPreview tinkering

In [ ]:
_GFUserId = getSurveysOfBiologists(gform)[localplayerguidkey].iloc[2]
#sample = gform[gform[localplayerguidkey] == _GFUserId]
_RMUserId = unsafeGetRedMetricsGUIDFormat(_GFUserId)

In [ ]:
_rmDF[_rmDF['sessionId'] == _sessionId]['type'].value_counts()

In [ ]:
_rmDF = df152
sessions = getAllSessionsOfUser( _rmDF, _RMUserId, True )
_sessionId = sessions['sessionId'].iloc[0]

# for per-session, manual analysis
#def getSessionDataPreview( _sessionId, _rmDF ):

_logs = _rmDF[_rmDF['sessionId'] == _sessionId]

_timedEvents = _logs['userTime']
_timedEvents = _timedEvents.map(lambda t: pd.to_datetime(t, utc=True))
_timedEvents = _timedEvents.sort_values()
_platform = _logs['customData.platform'].dropna()[0]
_events = _logs['type'].value_counts()
result = {
    'first' : _timedEvents.iloc[0],
    'last' : _timedEvents.iloc[-1],
    'platform' : _platform,
    'events' : _events
}
print(result)

In [ ]:
events, first, last, platform, = result.values()

In [ ]:
first, last, platform, events

#### getUserDataPreview tinkering

In [ ]:
_GFUserId = getSurveysOfBiologists(gform)[localplayerguidkey].iloc[2]
#sample = gform[gform[localplayerguidkey] == _GFUserId]
_RMUserId = unsafeGetRedMetricsGUIDFormat(_GFUserId)

In [ ]:
events, first, last, platform

In [ ]:
events

In [ ]:
sdp = getSessionDataPreview(_sessionId, _rmDF = _rmDF)

In [ ]:
sdp

In [ ]:
#_GFUserId = getRandomGFormGUID()
#_RMUserId = unsafeGetRedMetricsGUIDFormat(_GFUserId)
_rmDF = df152
scoreLabel = 'score'

# for per-user, manual analysis
#def getUserDataPreview( _RMUserId, _rmDF = df152 ):
_GFUserId = unsafeGetGFormGUIDFormat(_RMUserId)
result = pd.DataFrame(
    columns = [_RMUserId]
)

#    [ ] RM
result.loc['REDMETRICS ANALYSIS'] = ' '
#      [ ] sessions count
sessions = getAllSessionsOfUser( _rmDF, _RMUserId, True )
result.loc['sessions', _RMUserId] = len(sessions)
#      [ ] first event date
result.loc['firstEvent', _RMUserId] = getFirstEventDate( _RMUserId )
#      [ ] time played
#      [ ] dates played
#      [ ] first played, last played
sessionIds = sessions['sessionId']
for _sessionIdIndex in range(0, len(sessions['sessionId'])):
    _sessionId = sessionIds.iloc[_sessionIdIndex]
    sdp = getSessionDataPreview(_sessionId, _rmDF = _rmDF)

    result.loc['session' + str(_sessionIdIndex) + ' platform',_RMUserId] = sdp['platform']
    result.loc['session' + str(_sessionIdIndex) + ' first',_RMUserId] = sdp['first']
    result.loc['session' + str(_sessionIdIndex) + ' last',_RMUserId] = sdp['last']
    result.loc['session' + str(_sessionIdIndex) + ' events',_RMUserId] = str(sdp['events'])
#      [ ] best chapter
#      [ ] counts of events: deaths, crafts,...

#    [ ] GF
result.loc['GFORM ANALYSIS'] = ' '
#      [ ] score(s)
score = getScore( _RMUserId )
for _temporality in score.columns:
    _score = score.loc[scoreLabel,_temporality]
    if(len(_score)>0):
        if(_temporality == 'before'):
            _score = _score[len(_score)-1]
        else:
            _score = _score[0]
    else:
        _score = np.nan
    result.loc[scoreLabel+_temporality,_RMUserId] = _score
#        [ ] progression
#      [ ] demographics
result.loc[scoreLabel+'s',_RMUserId] = str(score.values)

gfDataPreview = getGFormDataPreview(_GFUserId, gform)
features = {1: 'date', 2: 'temporality RM', 3: 'temporality GF', 4: 'score', 5: 'genderAge'}
for key in gfDataPreview:
    for featureKey in features:
        result.loc[key + ' ' + features[featureKey]] = str(gfDataPreview[key][features[featureKey]])
    index = 0
    for match in gfDataPreview[key]['demographic matches']:
        result.loc[key + ' demographic match ' + str(index)] = repr(match)
        index += 1
     
result

In [ ]:
answerTemporalities

In [ ]:
getUserDataPreview(undefinedId)

In [ ]:
for undefinedId in gform[gform['Temporality'] == answerTemporalities[2]][localplayerguidkey]:
    getUserDataPreview(undefinedId)

# Checks on user metrics
<a id=checkusermetrics />

### Sequence of actions

    sandbox
    crafting
    equip device
    unequip device
    add PCONS
    add 6
    add Ampicillin
    add T
    > auto craft
    > auto equip
    remove T
    > auto unequip
    add T
    > auto equip
    add 12
    > auto craft
    > auto equip
    add 6
    > auto equip
    exit crafting
    dies
    > auto unequip
    set language to english

In [ ]:
rdfcrafttest = pd.read_csv("../../data/2017-10-10.craft-test.csv")
rdfcrafttest = getNormalizedRedMetricsCSV(rdfcrafttest)
rdfcrafttest

In [ ]:
craftEventsColumns

In [ ]:
craftEventsColumns['column']['equip']

In [ ]:
type(craftEventCodes)

In [ ]:
test = np.unique(np.concatenate((perSessionRelevantColumns, [craftEventsColumns['column']['equip']])))
test

In [ ]:
# user 344 adds
#'"e2f8d5e4-cccd-4d1a-909b-c9c92f6b83c1"'
# one of its sessions
# fab3ea03-6ff1-483f-a90a-74ff47d0b556
#
# user 22 adds
#'"8d352896-a3f1-471c-8439-0f426df901c1"'
#

# session test
craftSessionTest = getSectionsCraftEvents('equip', "fab3ea03-6ff1-483f-a90a-74ff47d0b556")
# user test
craftUserTest = getUserSectionsCraftEvents('equip', '"e2f8d5e4-cccd-4d1a-909b-c9c92f6b83c1"')
# user count test
craftUserTestCount = getUserSectionsCraftEventsTotal('equip', '"e2f8d5e4-cccd-4d1a-909b-c9c92f6b83c1"')
craftUserTestCount

In [ ]:
print("craftSessionTest=" + str(craftSessionTest))
print("craftUserTest=" + str(craftUserTest))
print("craftUserTestCount=" + str(craftUserTestCount))

In [ ]:
columnName = craftEventsColumns['column']['equip']
columnName

In [ ]:
result = list([])
for entry in df152[columnName]:
    if not pd.isnull(entry):
        result.append(entry)
result

In [ ]:
#df152[columnName].notnull()

In [ ]:
sectionsEvents = pd.DataFrame(0, columns=eventSectionsCountColumns, index=range(0))
sectionsEvents

In [ ]:
#events = df152[df152['type']==eventType and not df152[craftEventsColumns['column'][eventCode]].isnull()].loc[:,perSessionRelevantColumns]

#### getUserDataVector tinkering

In [ ]:
testUser = unsafeGetRedMetricsGUIDFormat(getRandomGFormGUID())
print(testUser)
#testResult = getUserDataVector(testUser)
#testResult

In [ ]:
userId = unsafeGetRedMetricsGUIDFormat(getRandomGFormGUID())
#userId = '"1f27519a-971f-4e39-bac7-9920bfc4b05b"' #undefined temporality
print(userId)
isRMId = True
_source = correctAnswers
_rmDF = df152

#def getUserDataVector( userId, isRMId = True, _source = [], _rmDF = df152 ):
rmUserId = userId
if(not isRMId):
    rmUserId = unsafeGetRedMetricsGUIDFormat(userId)

sessionsList = getAllSessionsOfUser( _rmDF, rmUserId, True )

columnName = str(rmUserId)

data = pd.DataFrame(0, columns=[columnName],index=userDataVectorIndex)

score = getScore( rmUserId )
for _temporality in score.columns:
    _score = score.loc[scoreLabel,_temporality]
    if(len(_score)>0):
        if(_temporality == 'before'):
            _score = _score[len(_score)-1]
        else:
            _score = _score[0]
    else:
        _score = np.nan
    data.loc[scoreLabel+_temporality,columnName] = _score

data.loc['sessionsCount',columnName] = countSessions( rmUserId, False, sessionsList, _rmDF = _rmDF)

for eventName in simpleEvents:
    if eventName in craftEventCodes:
        data.loc[eventName,columnName] = getUserCraftEventsTotal(eventName, rmUserId, sessionsList)
    else:
        data.loc[eventName,columnName] = getUserEventsTotal(eventName, rmUserId, sessionsList)
    #elif eventName in noSectionEventCodes:
    #    data.loc[eventName,columnName] = getUserEventsTotal(eventName, rmUserId, sessionsList)
    #else:
    #    data.loc[eventName,columnName] = getUserSectionsEventsTotal(eventName, rmUserId, sessionsList)

# time spent on each chapter
times = getCheckpointsTimesUser(rmUserId)

for chapter in timedSectionsIndex:
    data.loc[chapter,columnName] = times.loc[chapter,"firstCompletionDuration"].total_seconds()
    
if(len(_source) != 0):
    gfUserId = unsafeGetGFormGUIDFormat(rmUserId)
    gformLine = gform[gform[localplayerguidkey] == gfUserId]
    afters = gformLine[gformLine['Temporality'] == 'after']
    if(len(afters) > 0):
        gformLine = afters.iloc[0]
    else:
        befores = gformLine[gformLine['Temporality'] == 'before']
        if(len(befores) > 0):
            gformLine = befores.iloc[len(befores)-1]
        else:
            gformLine = gformLine.iloc[len(gformLine)-1]

    # add data from the gform: binary score on each question
    gformData = getBinarized(gformLine, _source = _source)
    
    for question in gformData.index:
        data.loc[question,columnName] = gformData.loc[question]
    
data

In [ ]:
times

In [ ]:
eventName

In [ ]:
getUserSectionsEvents( 'start', userId, sessionsList )

In [ ]:
perSession = perSession[perSession['section'].str.startswith('tutorial', na=False)]